# Element Parameter Detection


## Setup


In [ ]:
%load_ext autoreload

import numpy as np
import math
import random
import os

import torch
import pickle

from tqdm.notebook import tqdm
from ipywidgets import interact 
import gc
import open3d as o3d

from src.elements import *
from src.ifc import *
from src.preparation import *
from src.visualisation import *
from src.chamfer import *
from src.utils import *
from src.morph import *

random.seed = 42


In [ ]:
!pip install compas

In [ ]:
from pytorch3d.ops import utils as pytorch3d_utils
from pytorch3d.ops.points_normals import _disambiguate_vector_directions
from pytorch3d.ops.knn import knn_points
from typing import Tuple, TYPE_CHECKING, Union
from pytorch3d.common.workaround import symeig3x3


def get_point_covariances_relative(
    points_padded: torch.Tensor,
    targets_padded: torch.Tensor,
    num_points_per_cloud: torch.Tensor,
    neighborhood_size: int,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Computes the per-point covariance matrices by of the 3D locations of
    K-nearest neighbors of each point.

    Args:
        **points_padded**: Input point clouds as a padded tensor
            of shape `(minibatch, num_points, dim)`.
        **num_points_per_cloud**: Number of points per cloud
            of shape `(minibatch,)`.
        **neighborhood_size**: Number of nearest neighbors for each point
            used to estimate the covariance matrices.

    Returns:
        **covariances**: A batch of per-point covariance matrices
            of shape `(minibatch, dim, dim)`.
        **k_nearest_neighbors**: A batch of `neighborhood_size` nearest
            neighbors for each of the point cloud points
            of shape `(minibatch, num_points, neighborhood_size, dim)`.
    """
    # get K nearest neighbor idx for each point in the point cloud
    k_nearest_neighbors = knn_points(
        targets_padded,
        points_padded,
        lengths1=num_points_per_cloud,
        lengths2=num_points_per_cloud,
        K=neighborhood_size,
        return_nn=True,
    ).knn
    # obtain the mean of the neighborhood
    pt_mean = k_nearest_neighbors.mean(2, keepdim=True)
    # compute the diff of the neighborhood and the mean of the neighborhood
    central_diff = k_nearest_neighbors - pt_mean
    # per-nn-point covariances
    per_pt_cov = central_diff.unsqueeze(4) * central_diff.unsqueeze(3)
    # per-point covariances
    covariances = per_pt_cov.mean(2)

    return covariances, k_nearest_neighbors


# calculate eigen values and eigen vectors relative to points from a second point cloud
def estimate_pointcloud_local_coord_frames_relative(
    pointclouds: Union[torch.Tensor, "Pointclouds"],
    targets: Union[torch.Tensor, "Pointclouds"],
    neighborhood_size: int = 50,
    disambiguate_directions: bool = True,
    *,
    use_symeig_workaround: bool = True,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Estimates the principal directions of curvature (which includes normals)
    of a batch of `pointclouds`.

    The algorithm first finds `neighborhood_size` nearest neighbors for each
    point of the point clouds, followed by obtaining principal vectors of
    covariance matrices of each of the point neighborhoods.
    The main principal vector corresponds to the normals, while the
    other 2 are the direction of the highest curvature and the 2nd highest
    curvature.

    Note that each principal direction is given up to a sign. Hence,
    the function implements `disambiguate_directions` switch that allows
    to ensure consistency of the sign of neighboring normals. The implementation
    follows the sign disabiguation from SHOT descriptors [1].

    The algorithm also returns the curvature values themselves.
    These are the eigenvalues of the estimated covariance matrices
    of each point neighborhood.

    Args:
      **pointclouds**: Batch of 3-dimensional points of shape
        `(minibatch, num_point, 3)` or a `Pointclouds` object.
      **neighborhood_size**: The size of the neighborhood used to estimate the
        geometry around each point.
      **disambiguate_directions**: If `True`, uses the algorithm from [1] to
        ensure sign consistency of the normals of neighboring points.
      **use_symeig_workaround**: If `True`, uses a custom eigenvalue
        calculation.

    Returns:
      **curvatures**: The three principal curvatures of each point
        of shape `(minibatch, num_point, 3)`.
        If `pointclouds` are of `Pointclouds` class, returns a padded tensor.
      **local_coord_frames**: The three principal directions of the curvature
        around each point of shape `(minibatch, num_point, 3, 3)`.
        The principal directions are stored in columns of the output.
        E.g. `local_coord_frames[i, j, :, 0]` is the normal of
        `j`-th point in the `i`-th pointcloud.
        If `pointclouds` are of `Pointclouds` class, returns a padded tensor.

    References:
      [1] Tombari, Salti, Di Stefano: Unique Signatures of Histograms for
      Local Surface Description, ECCV 2010.
    """

    points_padded, num_points = pytorch3d_utils.convert_pointclouds_to_tensor(pointclouds)
    targets_padded, target_num_points = pytorch3d_utils.convert_pointclouds_to_tensor(targets)

    ba, N, dim = points_padded.shape
    if dim != 3:
        raise ValueError(
            "The pointclouds argument has to be of shape (minibatch, N, 3)"
        )

    if (num_points <= neighborhood_size).any():
        raise ValueError(
            "The neighborhood_size argument has to be"
            + " >= size of each of the point clouds."
        )

    # undo global mean for stability
    # TODO: replace with tutil.wmean once landed
    pcl_mean = points_padded.sum(1) / num_points[:, None]
    points_centered = points_padded - pcl_mean[:, None, :]
    targets_centered = targets_padded - pcl_mean[:, None, :]

    # get the per-point covariance and nearest neighbors used to compute it
    cov, knns = get_point_covariances_relative(points_centered, targets_centered, num_points, neighborhood_size)

    # get the local coord frames as principal directions of
    # the per-point covariance
    # this is done with torch.symeig / torch.linalg.eigh, which returns the
    # eigenvectors (=principal directions) in an ascending order of their
    # corresponding eigenvalues, and the smallest eigenvalue's eigenvector
    # corresponds to the normal direction; or with a custom equivalent.
    if use_symeig_workaround:
        curvatures, local_coord_frames = symeig3x3(cov, eigenvectors=True)
    else:
        curvatures, local_coord_frames = torch.linalg.eigh(cov)

    # disambiguate the directions of individual principal vectors
    if disambiguate_directions:
        # disambiguate normal
        n = _disambiguate_vector_directions(
            points_centered, knns, local_coord_frames[:, :, :, 0]
        )
        # disambiguate the main curvature
        z = _disambiguate_vector_directions(
            points_centered, knns, local_coord_frames[:, :, :, 2]
        )
        # the secondary curvature is just a cross between n and z
        y = torch.cross(n, z, dim=2)
        # cat to form the set of principal directions
        local_coord_frames = torch.stack((n, y, z), dim=3)

    return curvatures, local_coord_frames

### sphere morphing


In [2]:
# visualise a list of point clouds as an animation using open3d
# use ctrl+c to copy and ctrl+v to set camera and zoom inside visualiser
def create_point_cloud_animation(cloud_list, loss_func, save_image=False, colours=None):
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    cloud = cloud_list[0]
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(cloud)
    if colours is not None:
        point_cloud.colors = o3d.utility.Vector3dVector(colours[0])
    vis.add_geometry(point_cloud)
    stops = [9,39,99,299,999]

    for i in range(len(cloud_list)):
        time.sleep(0.01 + 0.05/(i/10+1))
        cloud = cloud_list[i]
        point_cloud.points = o3d.utility.Vector3dVector(cloud)
        if colours is not None:
            point_cloud.colors = o3d.utility.Vector3dVector(colours[i])
        vis.update_geometry(point_cloud)
        vis.poll_events()
        vis.update_renderer()
        if save_image and i in stops:
            vis.capture_screen_image("sphere/" + loss_func + str(i) + ".jpg", do_render=True)
    vis.destroy_window()

    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Info)


In [ ]:
# cld1_name = "sphere/chair.pcd"
# loss_func = "chamfer"
# run_morph(cld1_name, loss_func)

In [ ]:
from pytorch3d.ops import points_normals
from eindex import eindex
import einops

# calculate chamfer loss based on local curvature
def calc_curvature_loss_tensor(x, y, k=32, return_assignment=False):
    chamferDist = ChamferDistance()
    eps = 0.00001

    # add a loss term for mismatched pairs
    nn = chamferDist(
        x, y, bidirectional=True, return_nn=True
    )

    eig_vals_x, eig_vects_x = estimate_pointcloud_local_coord_frames_relative(
        x, y, neighborhood_size=k)

    eig_vals_y, eig_vects_y = estimate_pointcloud_local_coord_frames_relative(
        y, x, neighborhood_size=k)

    corresponding_y_vals = eindex(eig_vals_y, torch.squeeze(nn[0].idx, dim=-1), "batch [batch points] eigenvalues")
    corresponding_x_vals = eindex(eig_vals_x, torch.squeeze(nn[1].idx, dim=-1), "batch [batch points] eigenvalues")
    
    corresponding_y_vects = eindex(eig_vects_y, torch.squeeze(nn[0].idx, dim=-1), "batch [batch points] eigenvalues eigenvects")
    corresponding_x_vects = eindex(eig_vects_x, torch.squeeze(nn[1].idx, dim=-1), "batch [batch points] eigenvalues eigenvects")
    #print("x", x.shape, nn[0].dists.shape, nn[0].dists.shape, torch.squeeze(nn[1].idx, dim=-1).shape)
    
    #print("sample eigen", eig_vects_x[0,:4,:,:], eig_vals_x[0,:4])

    eigen_val_dist_y = torch.sum(torch.square(eig_vals_x - corresponding_y_vals))
    eigen_val_dist_x = torch.sum(torch.square(eig_vals_y - corresponding_x_vals))
    
    # calculate dot product between eigenvectors
    dot_product_y = einops.einsum(eig_vects_x, corresponding_y_vects, "b n p q, b n p q -> b n p")
    # ignore direction and take absolute value
    dot_product_y = torch.sum(1. - torch.abs(dot_product_y))
    dot_product_x = einops.einsum(eig_vects_y, corresponding_x_vects, "b n p q, b n p q -> b n p")
    print("dot", (1. - torch.abs(dot_product_x))[0,:3])
    # ignore direction and take absolute value
    dot_product_x = torch.sum(1. - torch.abs(dot_product_x))
    

    #corresponding_y_points = eindex(x, torch.squeeze(nn[1].idx, dim=-1), "batch [batch points] xyz")
    #print("corresponding shape", corresponding_y_vals.shape, corresponding_y_vects.shape)
    #print("mean", eigen_val_dist_x)
    
    #curvature_loss = (dot_product_x + dot_product_y)/1000 + (eigen_val_dist_x + eigen_val_dist_y)*1000
    curvature_loss = (dot_product_x + dot_product_y)/1000
    #print("curvature loss", curvature_loss)

    #print("eig_vals_x", eig_vals_x.shape, eig_vects_x.shape, "eig_vals_y", eig_vals_y.shape, eig_vects_y.shape)

    bidirectional_dist = torch.sum(nn[0].dists[:,:,0]) + torch.sum(nn[1].dists[:, :, 0])
    print("d", torch.sum(nn[1].dists[:,:,0]).item(), torch.sum(nn[0].dists[:,:,0]).item())

    return curvature_loss + bidirectional_dist, None

In [ ]:
import torch

# Example input tensors
b, n, k = 2, 4, 3
values = torch.rand(b, n, k)  # Random values tensor of shape [b, n, k]
indices = torch.randint(0, n, (b, n, k))  # Random indices tensor of shape [b, n, k] with values from 0 to n-1

# Initialize the output tensor Z of shape [b, n] with zeros
Z = torch.zeros(b, n)

# Use scatter_add to sum values according to indices
for i in range(b):
    Z[i].scatter_add_(0, indices[i].flatten(), values[i].flatten())

print("Values tensor:\n", values)
print("Indices tensor:\n", indices)
print("Resultant tensor Z:\n", Z)


In [ ]:
# measure the distance between a point and its correspondence's corresponding point
def calc_dcd_correspondence_tensor(x, y, k=8, return_assignment=False, return_dists=False):

    chamferDist = ChamferDistance()
    nn = chamferDist(
        x,
        y,
        bidirectional=True,
        return_nn=True,
        k=k
    )

    eps = 0.00001
    batch_size, point_count, _ = x.shape

    softmaxed_0 = torch.nn.functional.softmax(1/(nn[0].dists+eps), dim=-1)
    softmaxed_1 = torch.nn.functional.softmax(1/(nn[1].dists+eps), dim=-1)

    point_weights_1 = torch.zeros(batch_size, point_count, dtype=torch.float64).cuda()
    for i in range(batch_size):
        point_weights_1[i].scatter_add_(0, nn[0].idx[i].flatten(), softmaxed_0[i].flatten())

    point_weights_0 = torch.zeros(batch_size, point_count, dtype=torch.float64).cuda()
    for i in range(batch_size):
        point_weights_0[i].scatter_add_(0, nn[1].idx[i].flatten(), softmaxed_1[i].flatten())

    #print("w", point_weights_1.shape)

    # Extract necessary tensors from nn for better readability
    idx_0 = nn[0].idx[:, :, 0]
    dists_0 = nn[0].dists[:, :, 0]
    idx_1 = nn[1].idx[:, :, 0]
    dists_1 = nn[1].dists[:, :, 0]

    # Use advanced indexing to gather point weights and calculate weighted distances
    corresponding_weights_0 = point_weights_1.unsqueeze(2).repeat(1,1,k).gather(1, nn[0].idx)
    corresponding_weights_1 = point_weights_0.unsqueeze(2).repeat(1,1,k).gather(1, nn[1].idx)
    
    _, i0 = torch.min(corresponding_weights_0, dim=2)
    _, i1 = torch.min(corresponding_weights_1, dim=2)
    
    min_dist_1 = torch.gather(nn[1].dists, 2, i1.unsqueeze(2).repeat(1,1,k))[:, :, 0]
    min_dist_0 = torch.gather(nn[0].dists, 2, i0.unsqueeze(2).repeat(1,1,k))[:, :, 0]

    dcd = torch.sum(min_dist_1) + torch.sum(min_dist_0)
    dcd = dcd / (batch_size * point_count)

    # corresponding_weights_1 = point_weights_0.gather(1, nn[1].idx)
    
    print("corres", corresponding_weights_0.shape, i0.shape, min_dist_0.shape)

    bidirectional_dist = torch.sum(nn[0].dists[:,:,0]) + torch.sum(nn[1].dists[:,:,0])
    bidirectional_dist = bidirectional_dist / (batch_size * point_count)
    
    print("dcd", dcd.item(), bidirectional_dist.item())
    
    return dcd

    # weighted_dist = torch.sum(weighted_distances_0) + torch.sum(weighted_distances_1)
    # weighted_dist = weighted_dist / (batch_size * point_count) *10
    
    # weights = torch.sum(torch.square(point_weights_0)) + torch.sum(torch.square(point_weights_1))
    # weights = weights / (batch_size * point_count) *0.1

    # print("wd", weighted_dist.item(), bidirectional_dist.item(),
    #       "weights", weights.item(),
    #       "std", torch.std(point_weights_0).item(), torch.std(point_weights_1).item(),
    #       "sum", torch.sum(point_weights_0).item(), torch.sum(point_weights_1).item())

    # return bidirectional_dist + weights



In [ ]:
cld1_name = "sphere/plane1.pcd"
cld2_name = "sphere/chair.pcd"

cuda = torch.device("cuda")
cld1 = np.array(o3d.io.read_point_cloud(cld1_name).points)
cld2 = np.array(o3d.io.read_point_cloud(cld2_name).points)
pcd1_tensor = torch.tensor([cld1], device=cuda)
pcd2_tensor = torch.tensor([cld2], device=cuda)

l = calc_dcd_correspondence_tensor(pcd1_tensor, pcd2_tensor)
print(l)

In [ ]:
%autoreload 2

# visualise animation
cld1_name = "sphere/plane1.pcd"
visualise = False
#loss_funcs = [ "chamfer", "emd", "balanced", "reverse", "single"]
#loss_funcs = [ "balanced", "single"]
loss_funcs = [ "cyclic"]
for loss_func in loss_funcs:
    print(loss_func)
    run_morph(cld1_name, loss_func, lr=.01)
    if visualise:
        with open("sphere/" + loss_func + ".pkl", "rb") as f:
            morphed = pickle.load(f)
        colours = visualise_density(morphed, 'plasma_r')
        with open("sphere/" + loss_func + "_dens.pkl", "wb") as f:
            pickle.dump(colours, f)
        #create_point_cloud_animation(cloud_list, loss_func)
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
def view_density(loss_func): 
    with open("sphere/" + loss_func + ".pkl", "rb") as f:
        morphed = pickle.load(f)
    with open("sphere/" + loss_func + "_dens.pkl", "rb") as f:
        colours = pickle.load(f)

    create_point_cloud_animation(morphed, loss_func, True, colours[:,:,:3])

interact(view_density, loss_func=["cyclic", "chamfer", "curvature", "density", "balanced", "infocd", "single", "reverse", "emd", "direct"]);


In [ ]:
m = nn.Softmin(dim=1)
input = torch.randn(2, 3)
output = m(input)
print(input)
print(output)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# visualise animation
loss_func = "emd"
with open("sphere/" + loss_func + ".pkl", "rb") as f:
    morphed = pickle.load(f)
print(morphed.shape, loss_func)
cloud_list = [m for m in morphed]
#create_point_cloud_animation(cloud_list, loss_func)


In [ ]:
colours = visualise_density(morphed, 'plasma_r')
with open("sphere/" + loss_func + "_dens.pkl", "wb") as f:
    pickle.dump(colours, f)

### Batch optimisation


In [ ]:
# downsample
shapenet_path = "../experiments/ICCV2023-HyperCD/ShapeNetCompletion/test/complete/"
downsampled_path = "../experiments/ICCV2023-HyperCD/ShapeNetCompletion/downsample/"
# folders = os.listdir(shapenet_path)
# choices = np.random.choice(len(points), 4096)

# for fl in tqdm(folders):
#     if not os.path.exists(downsampled_path+fl):
#         os.mkdir(downsampled_path+fl)
        
#     files = os.listdir(shapenet_path + fl)
#     cloud =  o3d.geometry.PointCloud()
#     for cl in files:
#         points = np.array(o3d.io.read_point_cloud(shapenet_path + fl + "/" + cl).points)
#         points = points[choices]
#         cloud.points = o3d.utility.Vector3dVector(points)
#         o3d.io.write_point_cloud(downsampled_path+fl + "/" + cl, cloud)

In [ ]:
loss_func = "balanced"

cd, emd = sphere_morph_metrics(loss_func, downsampled_path, save=False)
print(cd[-1], emd[-1])

In [ ]:
loss_func = "balanced"
#loss_func = "emd"

sphere_morph_metrics(loss_func, downsampled_path)

In [ ]:
# plot losses on same axis
def plot_losses(losses, labels, title):
    x = np.arange(0, len(losses[0]))
    plt.figure(figsize=(30, 6))
    for i, loss in enumerate(losses):
        plt.plot(x, loss, label=labels[i])

    plt.xlabel("point cloud index")
    plt.ylabel("distance")
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
# create plots
loss_funcs = ["chamfer",]
chamfer_list, emd_list = [], []
for loss_func in loss_funcs:
    with open("sphere/" + loss_func + "_metrics.pkl", "rb") as f:
        chamfer, emd, assignments = pickle.load(f)
        chamfer_list.append(chamfer)
        emd_list.append(emd)
        
plot_losses(chamfer_list, loss_funcs, "chamfer")

ts(emd_list, loss_funcs, "EMD")
        
print(emd_list[-1], chamfer_list[-1])

In [ ]:
# load losses
loss_types = ["reverse", "chamfer", "emd", "pair"]
losses = []

for loss_func in loss_types:
    with open("sphere/loss_" + loss_func + ".pkl", "rb") as f:
        losses.append(pickle.load(f))
        
plot_losses(losses, loss_types, "loss function comparison")


#### consistency


In [ ]:
# measure conssistency between forward and backward correspondences for chamfer distance
# optionally compare against the ideal assignment, as measured by EMD
def measure_assignment_consistency(assignment, emd=None):
    reverse_assignment = torch.gather(assignment[0], 0, assignment[1])
    expected = torch.arange(assignment[0].shape[0], device=torch.device("cuda"))
    consistency = torch.sum(torch.eq(expected, reverse_assignment).long())
    print("consistency", consistency.item(), len(torch.unique(assignment[0])), len(torch.unique(assignment[1])))
    
    if emd is not None:
        #print(emd[:5], assignment[0][:5], assignment[1][:5])
        emd_consistency = torch.sum(torch.eq(emd, assignment[0]).long())
        #print("emd_consistency", emd_consistency.item(), len(torch.unique(emd)))


In [ ]:
# compare correspondences
#TODO: include consistency in top5 matches?
loss_func = "emd"
with open("sphere/assignments_" + "emd" + ".pkl", "rb") as f:
    emd_assignment = pickle.load(f)

with open("sphere/assignments_" + loss_func + ".pkl", "rb") as f:
    assignment = pickle.load(f)
    
for i, ass in enumerate(assignment):
    #measure_assignment_consistency(ass, emd_assignment[i][0])
    measure_assignment_consistency(ass)


In [ ]:
# find the morphed middle ground between two different clouds
cld1_name = "sphere/plane1.pcd"
cld2_name = "sphere/chair.pcd"

cuda = torch.device("cuda")
cld1 = np.array(o3d.io.read_point_cloud(cld1_name).points)
cld2 = np.array(o3d.io.read_point_cloud(cld2_name).points)
pcd1_tensor = torch.tensor([cld1], device=cuda)
pcd2_tensor = torch.tensor([cld2], device=cuda)

print(pcd1_tensor.shape, pcd2_tensor.shape)
loss, assignment = calc_emd(pcd1_tensor, pcd2_tensor, 0.05, 50)
assignment = assignment.detach().cpu().numpy()
print(assignment.shape)

matched_points = cld2[assignment[0]]
print(matched_points.shape)

morphed_points = (cld1 + matched_points)/2
morphed_cloud =  o3d.geometry.PointCloud()
morphed_cloud.points = o3d.utility.Vector3dVector(morphed_points)
o3d.io.write_point_cloud("sphere/chair_plane_morph.pcd", morphed_cloud)

### spherical projection

In [ ]:
import open3d as o3d
import numpy as np
from PIL import Image

def load_point_cloud(file_path):
    """
    Loads a point cloud from the specified file path.

    Args:
        file_path (str): Path to the point cloud file.

    Returns:
        o3d.geometry.PointCloud: The loaded point cloud.
    """
    pcd = o3d.io.read_point_cloud(file_path)
    if not pcd.has_points():
        raise ValueError("The point cloud has no points.")
    return pcd

def point_cloud_to_equirectangular(pcd, img_width=512, img_height=512):
    """
    Projects a point cloud onto a 2D equirectangular image.

    Args:
        pcd (o3d.geometry.PointCloud): The input point cloud.
        img_width (int): Width of the output image.
        img_height (int): Height of the output image.

    Returns:
        np.ndarray: The equirectangular image as a NumPy array.
    """
    # Extract points and colors
    points = np.asarray(pcd.points)
    has_colors = pcd.has_colors()
    if has_colors:
        colors = np.asarray(pcd.colors)
    else:
        # If colors are not available, use depth for grayscale intensity
        colors = None

    # Initialize the image with black pixels
    if has_colors:
        # For RGB image
        img = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    else:
        # For grayscale image
        img = np.zeros((img_height, img_width), dtype=np.uint8)

    # Initialize depth buffer with infinities
    depth_buffer = np.full((img_height, img_width), np.inf)

    # Compute spherical coordinates for each point
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arctan2(z, x)  # Longitude
    phi = np.arcsin(y / r)    # Latitude

    # Normalize angles to [0, 1] range for mapping to image pixels
    u = (theta + np.pi) / (2 * np.pi)  # Range: [0, 1]
    v = (np.pi/2 - phi) / np.pi        # Range: [0, 1]

    # Map to pixel coordinates
    pixel_x = (u * img_width).astype(np.int32)
    pixel_y = (v * img_height).astype(np.int32)

    # Ensure pixel indices are within image bounds
    pixel_x = np.clip(pixel_x, 0, img_width - 1)
    pixel_y = np.clip(pixel_y, 0, img_height - 1)

    # Iterate over all points to update the image and depth buffer
    for idx in range(points.shape[0]):
        x_pix = pixel_x[idx]
        y_pix = pixel_y[idx]
        depth = r[idx]
        if depth < depth_buffer[y_pix, x_pix]:
            depth_buffer[y_pix, x_pix] = depth
            if has_colors:
                color = (colors[idx] * 255).astype(np.uint8)
                img[y_pix, x_pix, :] = color
            else:
                # Map depth to intensity (closer points are brighter)
                # Normalize depth for visualization
                # Here, you might need to adjust the depth scaling based on your data
                max_depth = np.max(r)
                min_depth = np.min(r)
                if max_depth == min_depth:
                    intensity = 255
                else:
                    intensity = ((1 - (depth - min_depth) / (max_depth - min_depth)) * 255).astype(np.uint8)
                img[y_pix, x_pix] = intensity

    return img

def save_image(img_array, output_path):
    """
    Saves the image array to the specified path.

    Args:
        img_array (np.ndarray): The image data.
        output_path (str): Path to save the image.
    """
    img = Image.fromarray(img_array)
    img.save(output_path)
    #print(f"Image saved to {output_path}")

# Path to your point cloud file (e.g., .ply, .pcd)
point_cloud_path = "1d63eb2b1f78aa88acf77e718d93f3e1.pcd"
# Path to save the output image
output_image_path = "equirectangular_projection.png"

# Load point cloud
print("Loading point cloud...")
pcd = load_point_cloud(point_cloud_path)

# Generate equirectangular projection
print("Generating equirectangular projection...")
img_width = 512  # You can adjust the image size as needed
img_height = 512
img = point_cloud_to_equirectangular(pcd, img_width, img_height)

# Save the resulting image
print("Saving image...")
save_image(img, output_image_path)
print("Done.")


In [ ]:
import open3d as o3d
import numpy as np
from PIL import Image

def image_to_point_cloud(img_array, intensity_threshold=10):
    """
    Converts a 2D equirectangular image (using pixel intensity as depth) back into a 3D point cloud.
    Filters out points based on an intensity threshold to prevent far background points.

    Args:
        img_array (np.ndarray): The input equirectangular grayscale image.
        intensity_threshold (int): The minimum intensity value to consider a valid point.

    Returns:
        o3d.geometry.PointCloud: The reconstructed 3D point cloud.
    """
    img_height, img_width = img_array.shape[:2]
    point_cloud = []

    # Assume the depth is normalized between some min and max values
    max_depth = 10.0  # Adjust this value as needed
    min_depth = 0.1   # Minimum depth

    for y in range(img_height):
        for x in range(img_width):
            intensity = img_array[y, x]
            
            # Apply the intensity threshold to filter out background
            if intensity < intensity_threshold:
                continue  # Skip points with low intensity (likely background)

            # Normalize intensity to [0, 1]
            normalized_intensity = intensity / 255.0

            # Map normalized intensity to depth range
            depth = min_depth + (1 - normalized_intensity) * (max_depth - min_depth)

            # Recover spherical coordinates from pixel coordinates
            u = x / img_width  # Normalized longitude
            v = y / img_height  # Normalized latitude
            theta = u * 2 * np.pi - np.pi  # Longitude in radians
            phi = np.pi/2 - v * np.pi       # Latitude in radians

            # Convert spherical coordinates back to Cartesian coordinates
            x_coord = depth * np.cos(phi) * np.cos(theta)
            y_coord = depth * np.sin(phi)
            z_coord = depth * np.cos(phi) * np.sin(theta)
            #print(x_coord, y_coord, z_coord)

            # Append the point to the point cloud
            point_cloud.append((x_coord, y_coord, z_coord))

    # Create the Open3D point cloud object
    pcd = o3d.geometry.PointCloud()
    points = np.array(point_cloud)
    pcd.points = o3d.utility.Vector3dVector(points)

    return pcd

def load_grayscale_image(image_path):
    """
    Loads a grayscale image from the specified path.

    Args:
        image_path (str): Path to the equirectangular grayscale image file.

    Returns:
        np.ndarray: The loaded image array.
    """
    img = Image.open(image_path).convert("L")  # Convert image to grayscale
    img_array = np.array(img)
    print(img_array.shape)
    return img_array

def save_point_cloud(pcd, output_path):
    """
    Saves the point cloud to a file.

    Args:
        pcd (o3d.geometry.PointCloud): The point cloud to save.
        output_path (str): Path to save the point cloud file.
    """
    o3d.io.write_point_cloud(output_path, pcd)
    print(f"Point cloud saved to {output_path}")

# Path to the equirectangular grayscale image
image_path = "equirectangular_projection.png"
output_point_cloud_path = "reconstructed_point_cloud.ply"

# Load the grayscale image
print("Loading grayscale image...")
img_array = load_grayscale_image(image_path)

# Reconstruct the point cloud from the grayscale image
print("Reconstructing point cloud...")
pcd = image_to_point_cloud(img_array, intensity_threshold=10)

# Save the reconstructed point cloud
print("Saving point cloud...")
save_point_cloud(pcd, output_point_cloud_path)
print("Done.")



In [ ]:
image_path = "../experiments/2d_autoencoder/outputs/"
output_point_cloud_path = "../experiments/2d_autoencoder/outputs/pcd/"

for i in range(16):
    img_array = load_grayscale_image(image_path + f"{i}.png")
    pcd = image_to_point_cloud(img_array, intensity_threshold=10)
    save_point_cloud(pcd, output_point_cloud_path + f"{i}.pcd")


In [ ]:
def batch_image_to_point_cloud(images, intensity_threshold=10):
    """
    Converts a batch of 2D equirectangular images (using pixel intensity as depth) back into 3D point clouds.
    Filters out points based on an intensity threshold to prevent far background points.

    Args:
        images (tf.Tensor): A batch of input equirectangular grayscale images of shape (batch_size, height, width).
        intensity_threshold (int): The minimum intensity value to consider a valid point.

    Returns:
        point_clouds (tf.Tensor): The batch of reconstructed 3D point clouds, with shape (batch_size, num_points, 3).
    """
    batch_size, img_height, img_width = tf.shape(images)[0], tf.shape(images)[1], tf.shape(images)[2]
    
    # Generate grid of coordinates
    x = tf.linspace(0.0, 1.0, img_width)
    y = tf.linspace(0.0, 1.0, img_height)
    x_grid, y_grid = tf.meshgrid(x, y)
    
    # Compute longitude and latitude in radians
    theta = x_grid * 2.0 * tf.constant(np.pi) - tf.constant(np.pi)
    phi = tf.constant(np.pi/2) - y_grid * tf.constant(np.pi)
    
    # Reshape for broadcasting
    theta = tf.reshape(theta, (1, img_height, img_width))
    phi = tf.reshape(phi, (1, img_height, img_width))
    
    # Normalize intensities and calculate depth
    normalized_intensity = images / 255.0
    max_depth = 10.0
    min_depth = 0.1
    depth = min_depth + (1.0 - normalized_intensity) * (max_depth - min_depth)
    
    # Apply the intensity threshold
    mask = tf.greater_equal(images, intensity_threshold)
    depth = tf.where(mask, depth, tf.zeros_like(depth))
    
    # Convert spherical coordinates to Cartesian coordinates
    x_coord = depth * tf.cos(phi) * tf.cos(theta)
    y_coord = depth * tf.sin(phi)
    z_coord = depth * tf.cos(phi) * tf.sin(theta)
    
    # Stack the coordinates into point clouds
    point_clouds = tf.stack([x_coord, y_coord, z_coord], axis=-1)
    
    # Reshape to (batch_size, num_points, 3)
    point_clouds = tf.reshape(point_clouds, (batch_size, -1, 3))
    
    # Filter out zero-depth points (these are the masked-out points)
    point_clouds = tf.boolean_mask(point_clouds, tf.not_equal(tf.norm(point_clouds, axis=-1), 0), axis=1)

    return point_clouds

# Path to the equirectangular grayscale image
image_path = "equirectangular_projection.png"
output_point_cloud_path = "reconstructed_point_cloud.ply"

# Load the grayscale image
print("Loading grayscale image...")
img_array = load_grayscale_image(image_path)
img_tensor = tf.tensor([img_array])

# Reconstruct the point cloud from the grayscale image
print("Reconstructing point cloud...")
cld = batch_image_to_point_cloud(img_array, intensity_threshold=10)

# Create the Open3D point cloud object
pcd = o3d.geometry.PointCloud()
points = np.array(cld)
pcd.points = o3d.utility.Vector3dVector(points)

# Save the reconstructed point cloud
print("Saving point cloud...")
save_point_cloud(pcd, output_point_cloud_path)

In [ ]:
# HEALPix
import open3d as o3d
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt

def point_cloud_to_healpix(pcd, nside=512):
    """
    Projects a 3D point cloud into a 2D HEALPix image using pixel intensity as depth.

    Args:
        pcd (o3d.geometry.PointCloud): The input 3D point cloud.
        nside (int): The resolution of the HEALPix map (higher nside means higher resolution).

    Returns:
        np.ndarray: The resulting HEALPix map with depth encoded as pixel intensity.
    """
    # Extract points from the point cloud
    points = np.asarray(pcd.points)
    x, y, z = points[:, 0], points[:, 1], points[:, 2]

    # Calculate the spherical coordinates
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r)  # colatitude (0 to pi)
    phi = np.arctan2(y, x)    # longitude (0 to 2*pi)

    # Map spherical coordinates to HEALPix pixel indices
    pix_indices = hp.ang2pix(nside, theta, phi)

    # Initialize the HEALPix map
    healpix_map = np.zeros(hp.nside2npix(nside), dtype=np.float32)

    # Aggregate depth values (use the closest point depth for each pixel)
    for i, pix in enumerate(pix_indices):
        if healpix_map[pix] == 0 or r[i] < healpix_map[pix]:  # closer point
            healpix_map[pix] = r[i]

    # Normalize depth to intensity (0-255 range)
    healpix_map = np.log1p(healpix_map)  # Use log scale for better depth perception
    healpix_map = (healpix_map - np.min(healpix_map)) / (np.max(healpix_map) - np.min(healpix_map))
    healpix_map = (healpix_map * 255).astype(np.uint8)

    return healpix_map

def visualize_healpix(healpix_map, nside):
    """
    Visualizes the HEALPix map using a Mollweide projection.

    Args:
        healpix_map (np.ndarray): The HEALPix map to visualize.
        nside (int): The resolution of the HEALPix map.
    """
    hp.mollview(healpix_map, coord=['C'], title='HEALPix Projection of Point Cloud', cmap='gray')
    plt.show()

def main():
    # Load the point cloud
    point_cloud_path = "sphere/chair.pcd"
    print("Loading point cloud...")
    pcd = o3d.io.read_point_cloud(point_cloud_path)

    # Convert the point cloud to a HEALPix image
    nside = 512  # Adjust the resolution (nside) as needed
    print("Projecting point cloud to HEALPix...")
    healpix_map = point_cloud_to_healpix(pcd, nside=nside)

    # Visualize the HEALPix image
    print("Visualizing HEALPix projection...")
    visualize_healpix(healpix_map, nside)

if __name__ == "__main__":
    main()


In [ ]:
# create equirectangular dataset from PCN

root_path = "/home/haritha/documents/experiments/ICCV2023-HyperCD/ShapeNetCompletion/train/"
input_path = root_path + "partial/"

classes = os.listdir(input_path)

for cl in classes:
    print(cl)
    folders = os.listdir(input_path + cl)
    for fl in tqdm(folders):
        files = os.listdir(input_path + cl + "/" + fl)
        for fl2 in files:
            pcd = o3d.io.read_point_cloud(input_path + cl + "/" + fl + "/" + fl2)
            img = point_cloud_to_equirectangular(pcd)
            img_path = root_path + "2d_partial/" + cl + "/" + fl + "/"
            if not os.path.exists(img_path):
                os.makedirs(img_path)
            img_path = root_path + "2d_partial/" + cl + "/" + fl + "/" + fl2[:-4] + ".png"
            save_image(img, img_path)


In [ ]:

root_path = "/home/haritha/documents/experiments/ICCV2023-HyperCD/ShapeNetCompletion/train/"
input_path = root_path + "complete/"

classes = os.listdir(input_path)

for cl in classes:
    print(cl)
    files = os.listdir(input_path + cl)
    for fl in tqdm(files):
        pcd = o3d.io.read_point_cloud(input_path + cl + "/" + fl )
        img = point_cloud_to_equirectangular(pcd)
        img_path = root_path + "2d_complete/" + cl + "/"
        if not os.path.exists(img_path):
            os.makedirs(img_path)
        img_path = root_path + "2d_complete/" + cl + "/" + fl[:-4] + ".png"
        save_image(img, img_path)
